In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
import wbdata as wb
from contextlib import contextmanager
import pytz
from datetime import timedelta

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api_stooq
from workers.common import read_json
from workers.common import get_cookie
import tradeDB

In [3]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.jsonc"},
            "STOCK": {"file": "./assets/stock.jsonc"},
            "ETF": {"file": "./assets/etf.jsonc"},
            "COMODITIES": {"file": "./assets/comodities.jsonc"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json( # type: ignore
        SECTORS[k]["file"]
    )

In [4]:
dat=pd.read_csv("./dev/long_dates.txt")
tab = "STOCK"
db = './trader.sqlite'
dat['indexes'] = 'WIG20'
symbol=dat.loc[0,'indexes']

In [14]:
trade= tradeDB.Trader()

In [60]:
dat = pd.read_csv("./dev/long_dates.txt",
                  sep='\t',
                  header=None)
dat[6]=pd.to_datetime(dat.iloc[:,6])
dat[7]=pd.to_datetime(dat.iloc[:,7])
dat


,0,1,2,3,4,5,6,7
0,c969ae6fb5a7dc0226fc05c00769da0f,^KOSPI,KOSPI,KR,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
1,40384e26d189cdf2be512a55bab5166d,^NKX,NIKKEI 225,JP,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
2,e89c035fca88d7bbfa271e43a8c98dbf,^NZ50,NZX 50,NZ,NaN,NaN,2023-03-28 00:15:00,2023-03-28 00:15:00
3,eb0bba47d1e86aa9f8f2f1d2fed0eddf,^PSEI,PSEI,PH,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
4,446c0273531f939770986c7e709ab95e,^SET,SET,TH,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
5,326255747a51575ef0892d301a8abf85,^SHBS,SHANGHAI B-SHARE,CN,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
6,3695c91615f9a346cdae757d96024d15,^SHC,SHANGHAI COMPOSITE,CN,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
7,8e0463793d39a5a45297ed02dcfc9767,^SNX,SENSEX 30,IN,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
8,5f57aa25ca0d5047d31a1c0d189899f8,^STI,STRAITS TIMES,SG,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00
9,ae978476e86613a07aadc3bfdba0b6ac,^TWSE,TAIEX,TW,NaN,NaN,2023-03-27 00:00:00,2023-03-27 00:00:00


In [63]:
dat.apply(lambda series: [dt.strftime(x,format="%Y-%m-%d") for x in series]  # type: ignore
                    if pd.api.types.is_datetime64_dtype(series) else series)

,0,1,2,3,4,5,6,7
0,c969ae6fb5a7dc0226fc05c00769da0f,^KOSPI,KOSPI,KR,NaN,NaN,2023-03-27,2023-03-27
1,40384e26d189cdf2be512a55bab5166d,^NKX,NIKKEI 225,JP,NaN,NaN,2023-03-27,2023-03-27
2,e89c035fca88d7bbfa271e43a8c98dbf,^NZ50,NZX 50,NZ,NaN,NaN,2023-03-28,2023-03-28
3,eb0bba47d1e86aa9f8f2f1d2fed0eddf,^PSEI,PSEI,PH,NaN,NaN,2023-03-27,2023-03-27
4,446c0273531f939770986c7e709ab95e,^SET,SET,TH,NaN,NaN,2023-03-27,2023-03-27
5,326255747a51575ef0892d301a8abf85,^SHBS,SHANGHAI B-SHARE,CN,NaN,NaN,2023-03-27,2023-03-27
6,3695c91615f9a346cdae757d96024d15,^SHC,SHANGHAI COMPOSITE,CN,NaN,NaN,2023-03-27,2023-03-27
7,8e0463793d39a5a45297ed02dcfc9767,^SNX,SENSEX 30,IN,NaN,NaN,2023-03-27,2023-03-27
8,5f57aa25ca0d5047d31a1c0d189899f8,^STI,STRAITS TIMES,SG,NaN,NaN,2023-03-27,2023-03-27
9,ae978476e86613a07aadc3bfdba0b6ac,^TWSE,TAIEX,TW,NaN,NaN,2023-03-27,2023-03-27


In [54]:
pd.to_datetime(dat[6],format="%d",errors='raise')

0   2023-03-27 00:00:00
1   2023-03-27 00:00:00
2   2023-03-28 00:15:00
3   2023-03-27 00:00:00
4   2023-03-27 00:00:00
5   2023-03-27 00:00:00
6   2023-03-27 00:00:00
7   2023-03-27 00:00:00
8   2023-03-27 00:00:00
9   2023-03-27 00:00:00
Name: 6, dtype: datetime64[ns]

In [25]:
reload(sql)
currency_symbol = sql.get(tab='GEO',
                          get=['currency_code'],
                          search=dat['country'].unique(),
                          cols=['iso2'],
                          db_file=trade.db)['iso2']

currency_symbol = sql.get(tab='GEO',
                          get=['currency_code'],
                          search=['PL','DE'],
                          cols=['iso2'],
                          db_file=trade.db)['iso2']


PRAGMA foreign_keys = ON
pragma table_info(GEO)
PRAGMA foreign_keys = ON
SELECT currency_code FROM GEO WHERE iso2 LIKE 'PL' OR iso2 LIKE 'none'
PRAGMA foreign_keys = ON
pragma table_info(GEO)
PRAGMA foreign_keys = ON
SELECT currency_code FROM GEO WHERE iso2 LIKE 'PL' OR  iso2 LIKE 'DE' OR iso2 LIKE 'none'


In [79]:
currency_symbol

,currency_code
0,EUR
1,PLN


In [80]:
reload(api_stooq)
[api_stooq.get(from_date=dt.today(),
                end_date = dt.today(),
                symbol='USD'+s) 
for s in currency_symbol['currency_code']]

KeyboardInterrupt: 

In [118]:
reload(api_stooq)
symbol='USDPLN'
from_date=dt.today() -timedelta(1)
end_date = dt.today()-timedelta(1)
from_dateS, end_dateS = api_stooq.__str_date__(from_date, end_date)
url = f"https://stooq.com/q/d/?s={symbol}&d1={from_dateS}&d2={end_dateS}"
#data = api_stooq.__take_page__(url)

In [119]:
url

'https://stooq.com/q/d/?s=USDPLN&d1=20230317&d2=20230317'

In [77]:
from_date=dt.today()
end_date = dt.today()
from_date = from_date.astimezone(pytz.timezone('Canada/Mountain'))
from_date -= timedelta(1)
from_dateS = dt.strftime(from_date, '%Y%m%d')
end_date = end_date.astimezone(pytz.timezone('Canada/Mountain'))

In [74]:
if end_date > from_date:
    end_date = from_date
end_dateS = dt.strftime(end_date, '%Y%m%d')

In [110]:
max(1,(dt(2023,3,17).weekday()+6) % 7 -3)

1

In [115]:
8%7

1

Su-6-2
Mo-0-3
Tu-1-0

In [76]:
end_date

datetime.datetime(2023, 3, 20, 14, 24, 12, 12692, tzinfo=<DstTzInfo 'Canada/Mountain' MDT-1 day, 18:00:00 DST>)

In [78]:
end_date > from_date

True

In [60]:
end_date > a

TypeError: can't compare offset-naive and offset-aware datetimes

In [123]:
pd.DataFrame().empty

True